In [1]:
import os
import sys
import subprocess
from pathlib import Path
import glob
from tqdm import tqdm
import time
import pandas as pd
import numpy as np
import cv2
import ast
from typing import Dict, List, Tuple, Optional

# Add HSMR to path
sys.path.append('./HSMR')

# Configuration
GAVD_SEQUENCES_DIR = "./GAVD-sequences"
OUTPUT_DIR = "./GAVD-hsmr-params"
HSMR_SCRIPT = "./HSMR/exp/run_demo.py"
GAVD_DATA_DIR = "./GAVD/data"
CROPPED_VIDEOS_DIR = "./GAVD-cropped-sequences"

# Create output directories
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(CROPPED_VIDEOS_DIR, exist_ok=True)

print(f"✅ Setup complete!")
print(f"📁 Input directory: {GAVD_SEQUENCES_DIR}")
print(f"📁 Cropped videos directory: {CROPPED_VIDEOS_DIR}")
print(f"📁 Output directory: {OUTPUT_DIR}")
print(f"🎥 Total videos found: {len(list(Path(GAVD_SEQUENCES_DIR).glob('*.mp4')))}")

# set $env:PYOPENGL_PLATFORM = "pyglet" for HSMR
os.environ['PYOPENGL_PLATFORM'] = 'pyglet'

✅ Setup complete!
📁 Input directory: ./GAVD-sequences
📁 Cropped videos directory: ./GAVD-cropped-sequences
📁 Output directory: ./GAVD-hsmr-params
🎥 Total videos found: 1801


In [2]:
# Load GAVD dataset with bounding box information
def load_gavd_dataset() -> pd.DataFrame:
    """
    Load the complete GAVD dataset from CSV files.
    
    Returns:
        pd.DataFrame: Complete dataset with bounding box information
    """
    print("📊 Loading GAVD dataset...")
    all_dfs = []
    
    for i in range(1, 6):  # Parts 1-5
        filename = f'GAVD_Clinical_Annotations_{i}.csv'
        filepath = Path(GAVD_DATA_DIR) / filename
        if filepath.exists():
            print(f"  Loading {filename}...")
            df_part = pd.read_csv(filepath)
            all_dfs.append(df_part)
            print(f"    Shape: {df_part.shape}")
    
    if all_dfs:
        df_complete = pd.concat(all_dfs, ignore_index=True)
        print(f"✅ Complete dataset loaded: {df_complete.shape}")
        print(f"📼 Total unique sequences: {df_complete['seq'].nunique()}")
        return df_complete
    else:
        raise FileNotFoundError("No GAVD CSV files found!")

def parse_bbox_string(bbox_str: str) -> Dict:
    """
    Parse bounding box string to dictionary.
    
    Args:
        bbox_str (str): String representation of bounding box dict
        
    Returns:
        Dict: Parsed bounding box with keys: top, left, height, width
    """
    try:
        if pd.isna(bbox_str) or bbox_str == 'nan':
            return None
        # Use ast.literal_eval to safely parse the string representation of dict
        bbox_dict = ast.literal_eval(bbox_str)
        return bbox_dict
    except (ValueError, SyntaxError) as e:
        print(f"⚠️ Error parsing bbox: {bbox_str} - {e}")
        return None

def get_sequence_bbox_data(sequence_id: str, gavd_df: pd.DataFrame) -> Optional[pd.DataFrame]:
    """
    Get bounding box data for a specific sequence.
    
    Args:
        sequence_id (str): The sequence ID to lookup
        gavd_df (pd.DataFrame): The GAVD dataset
        
    Returns:
        pd.DataFrame or None: Sequence data with bounding boxes, sorted by frame number
    """
    sequence_data = gavd_df[gavd_df['seq'] == sequence_id].copy()
    
    if sequence_data.empty:
        print(f"⚠️ No bounding box data found for sequence: {sequence_id}")
        return None
    
    # Sort by frame number
    sequence_data = sequence_data.sort_values('frame_num').reset_index(drop=True)
    
    # Parse bounding box strings
    sequence_data['bbox_parsed'] = sequence_data['bbox'].apply(parse_bbox_string)
    
    return sequence_data

def crop_video_with_bbox(input_video_path: str, output_video_path: str, 
                        sequence_data: pd.DataFrame, padding: int = 20) -> bool:
    """
    Crop video using bounding box data from GAVD dataset.
    
    Args:
        input_video_path (str): Path to input video
        output_video_path (str): Path to save cropped video
        sequence_data (pd.DataFrame): Sequence data with bounding boxes
        padding (int): Extra padding around bounding box
        
    Returns:
        bool: Success status
    """
    try:
        # Open input video
        cap = cv2.VideoCapture(input_video_path)
        if not cap.isOpened():
            print(f"❌ Cannot open video: {input_video_path}")
            return False
        
        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        print(f"🎬 Input video: {total_frames} frames at {fps} FPS")
        print(f"📋 Sequence data: {len(sequence_data)} frames")
        
        # Create frame number to bbox mapping
        bbox_map = {}
        for _, row in sequence_data.iterrows():
            frame_num = row['frame_num']
            bbox = row['bbox_parsed']
            if bbox is not None:
                bbox_map[frame_num] = bbox
        
        if not bbox_map:
            print("❌ No valid bounding boxes found!")
            return False
        
        # Calculate consistent crop region from all bounding boxes
        all_tops = [bbox['top'] for bbox in bbox_map.values()]
        all_lefts = [bbox['left'] for bbox in bbox_map.values()]
        all_bottoms = [bbox['top'] + bbox['height'] for bbox in bbox_map.values()]
        all_rights = [bbox['left'] + bbox['width'] for bbox in bbox_map.values()]
        
        # Use the bounding box that encompasses all person positions
        crop_top = max(0, int(min(all_tops)) - padding)
        crop_left = max(0, int(min(all_lefts)) - padding)
        crop_bottom = int(max(all_bottoms)) + padding
        crop_right = int(max(all_rights)) + padding
        
        # Get video dimensions for validation
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        crop_bottom = min(crop_bottom, height)
        crop_right = min(crop_right, width)
        
        crop_width = crop_right - crop_left
        crop_height = crop_bottom - crop_top
        
        print(f"📐 Crop region: ({crop_left}, {crop_top}) to ({crop_right}, {crop_bottom})")
        print(f"📏 Crop size: {crop_width}x{crop_height}")
        
        # Setup output video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (crop_width, crop_height))
        
        if not out.isOpened():
            print(f"❌ Cannot create output video: {output_video_path}")
            cap.release()
            return False
        
        # FIXED: Handle frame number mismatch by processing ALL video frames
        # The sequence frames from GAVD may not correspond directly to video frame indices
        sequence_frames = set(sequence_data['frame_num'].values)
        min_frame = min(sequence_frames)
        max_frame = max(sequence_frames)
        
        print(f"🎯 GAVD frame range: {min_frame} to {max_frame}")
        print(f"🎯 Video frame range: 0 to {total_frames-1}")
        
        # Since the frame numbers don't match video indices, process all video frames
        # and use the bounding box information proportionally
        frame_idx = 0
        frames_written = 0
        
        # Create a sorted list of bounding boxes for interpolation
        sorted_bboxes = [(row['frame_num'], row['bbox_parsed']) for _, row in sequence_data.iterrows() 
                        if row['bbox_parsed'] is not None]
        sorted_bboxes.sort(key=lambda x: x[0])
        
        if not sorted_bboxes:
            print("❌ No valid bounding boxes found after sorting!")
            cap.release()
            out.release()
            return False
        
        print(f"📦 Using {len(sorted_bboxes)} bounding boxes for cropping")
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Use a consistent bounding box for all frames (could be improved with interpolation)
            # For now, use the first valid bounding box
            bbox = sorted_bboxes[0][1]  # Use first bbox
            
            # Dynamic crop based on current bbox (optional: interpolate between bboxes)
            # For consistency, we'll use the pre-calculated crop region
            cropped_frame = frame[crop_top:crop_bottom, crop_left:crop_right]
            out.write(cropped_frame)
            frames_written += 1
            
            frame_idx += 1
        
        # Cleanup
        cap.release()
        out.release()
        
        print(f"✅ Cropped video saved: {frames_written} frames written")
        
        # Verify output file
        if frames_written == 0:
            print("⚠️ Warning: No frames were written to output file!")
            return False
            
        return True
        
    except Exception as e:
        print(f"❌ Error cropping video: {e}")
        return False

# UPDATED function with bounding box cropping
def process_video_with_hsmr(video_path, output_dir, gavd_df=None, use_bbox_crop=True, 
                          verbose=True, show_output=True):
    """
    Process a single video file using HSMR to extract skeleton parameters.
    Optionally crops video using GAVD bounding box data first.
    
    Args:
        video_path (str): Path to the input video file
        output_dir (str): Directory to save the output .npy file
        gavd_df (pd.DataFrame): GAVD dataset with bounding box information
        use_bbox_crop (bool): Whether to crop video using bounding box data
        verbose (bool): Whether to print progress information
        show_output (bool): Whether to show real-time command output
    
    Returns:
        tuple: (success, output_file_path, error_message)
    """
    video_path = Path(video_path)
    output_dir = Path(output_dir)
    
    if not video_path.exists():
        return False, None, f"Video file not found: {video_path}"
    
    # Expected output file name (HSMR adds the model name prefix)
    expected_output = output_dir / f"HSMR-{video_path.stem}.npy"
    
    # Check if already processed
    if expected_output.exists():
        if verbose:
            print(f"⏭️ Skipping {video_path.name} - already processed")
        return True, expected_output, None
    
    # Determine which video file to process
    video_to_process = video_path
    
    # Crop video using bounding box if requested and data is available
    if use_bbox_crop and gavd_df is not None:
        sequence_id = video_path.stem  # Extract sequence ID from filename
        sequence_data = get_sequence_bbox_data(sequence_id, gavd_df)
        
        if sequence_data is not None:
            # Create cropped video path
            cropped_video_path = Path(CROPPED_VIDEOS_DIR) / f"{sequence_id}_cropped.mp4"
            
            if not cropped_video_path.exists():
                if verbose:
                    print(f"✂️ Cropping video using bounding box data...")
                
                success = crop_video_with_bbox(
                    str(video_path), 
                    str(cropped_video_path), 
                    sequence_data
                )
                
                if not success:
                    if verbose:
                        print(f"⚠️ Cropping failed, using original video")
                else:
                    video_to_process = cropped_video_path
                    if verbose:
                        print(f"✅ Using cropped video: {cropped_video_path}")
            else:
                video_to_process = cropped_video_path
                if verbose:
                    print(f"📁 Using existing cropped video: {cropped_video_path}")
        else:
            if verbose:
                print(f"⚠️ No bounding box data found for {sequence_id}, using original video")
    
    try:
        # Construct the command with CORRECT model path
        model_root = "./HSMR/data_inputs/released_models/HSMR-ViTH-r1d1"
        cmd = [
            "python", 
            HSMR_SCRIPT,
            "--input_path", str(video_to_process),
            "--output_path", str(output_dir),
            "--model_root", model_root
        ]
        
        if verbose:
            print(f"🎬 Processing: {video_to_process.name}")
            print(f"💻 Command: {' '.join(cmd)}")
            if show_output:
                print("📺 Real-time output:")
                print("=" * 80)
        
        # Run the command with real-time output
        start_time = time.time()
        
        if show_output and verbose:
            # Show real-time output
            process = subprocess.Popen(
                cmd,
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                text=True,
                bufsize=1,
                universal_newlines=True,
                cwd="."
            )
            
            # Print output in real-time
            output_lines = []
            for line in iter(process.stdout.readline, ''):
                if line.strip():  # Only print non-empty lines
                    print(f"📟 {line.rstrip()}")
                output_lines.append(line)
            
            process.stdout.close()
            return_code = process.wait()
            
            full_output = ''.join(output_lines)
            
        else:
            # Capture output without showing (for batch processing)
            result = subprocess.run(
                cmd, 
                capture_output=True, 
                text=True, 
                cwd="."
            )
            return_code = result.returncode
            full_output = result.stdout
            stderr_output = result.stderr
        
        end_time = time.time()
        
        if return_code == 0:
            if verbose:
                if show_output:
                    print("=" * 80)
                print(f"✅ Success! Processed in {end_time - start_time:.2f}s")
                print(f"📄 Output: {expected_output}")
            return True, expected_output, None
        else:
            if show_output and verbose:
                error_msg = f"Command failed with return code {return_code}\\nOutput: {full_output}"
            else:
                error_msg = f"Command failed with return code {return_code}\\nSTDOUT: {full_output}\\nSTDERR: {stderr_output if 'stderr_output' in locals() else 'N/A'}"
            
            if verbose:
                if show_output:
                    print("=" * 80)
                print(f"❌ Failed: {error_msg}")
            return False, None, error_msg
            
    except Exception as e:
        error_msg = f"Exception occurred: {str(e)}"
        if verbose:
            print(f"❌ Exception: {error_msg}")
        return False, None, error_msg

# Load the GAVD dataset
try:
    gavd_dataset = load_gavd_dataset()
    print("🔧 UPDATED function with bounding box cropping defined successfully!")
    print("📍 Model path: ./HSMR/data_inputs/released_models/HSMR-ViTH-r1d1")
    print("✂️ Bounding box cropping: ENABLED")
except Exception as e:
    print(f"⚠️ Could not load GAVD dataset: {e}")
    print("🔧 Function defined with bounding box cropping DISABLED")
    gavd_dataset = None


📊 Loading GAVD dataset...
  Loading GAVD_Clinical_Annotations_1.csv...
    Shape: (91624, 10)
  Loading GAVD_Clinical_Annotations_2.csv...


C:\Users\1nkas-Strix-4090-ll\AppData\Local\Temp\ipykernel_33824\1934443960.py:17: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_part = pd.read_csv(filepath)


    Shape: (91623, 10)
  Loading GAVD_Clinical_Annotations_3.csv...
    Shape: (91623, 10)
  Loading GAVD_Clinical_Annotations_4.csv...
    Shape: (91623, 10)
  Loading GAVD_Clinical_Annotations_5.csv...
    Shape: (91623, 10)
✅ Complete dataset loaded: (458116, 10)
📼 Total unique sequences: 1874
🔧 UPDATED function with bounding box cropping defined successfully!
📍 Model path: ./HSMR/data_inputs/released_models/HSMR-ViTH-r1d1
✂️ Bounding box cropping: ENABLED


In [3]:
# Get a list of all video files
video_files = list(Path(GAVD_SEQUENCES_DIR).glob('*.mp4'))
print(f"📼 Found {len(video_files)} video files")

# Select first 3 videos for sample processing
sample_videos = video_files[1:2]
print(f"🎯 Selected {len(sample_videos)} videos for sample processing:")
for i, video in enumerate(sample_videos, 1):
    print(f"  {i}. {video.name} ({video.stat().st_size / (1024*1024):.1f} MB)")

print("\\n" + "="*50)
print("🚀 Starting sample processing...")
print("="*50)


📼 Found 1801 video files
🎯 Selected 1 videos for sample processing:
  1. cljanb45y00083n6lmh1qhydd.mp4 (0.5 MB)
\n==================================================
🚀 Starting sample processing...


In [11]:
# # Test the FIXED cropping function
# print("🔄 Testing FIXED bounding box cropping (ignoring GAVD frame numbers)...")
# print("=" * 70)

# # Clean up any existing test files
# test_sequence = "cljanb45y00083n6lmh1qhydd"
# test_files = list(Path(CROPPED_VIDEOS_DIR).glob(f'{test_sequence}*'))
# for test_file in test_files:
#     test_file.unlink()
#     print(f"🗑️ Cleaned up: {test_file.name}")

# # Run the test
# test_bbox_cropping(test_sequence)


In [4]:
# Test bounding box cropping functionality with frame analysis
def test_bbox_cropping(sequence_id="cljanb45y00083n6lmh1qhydd"):
    """Test the bounding box cropping functionality on a specific sequence."""
    
    if gavd_dataset is None:
        print("❌ GAVD dataset not loaded. Cannot test bounding box cropping.")
        return
    
    print(f"🧪 Testing bounding box cropping for sequence: {sequence_id}")
    print("=" * 60)
    
    # Get sequence data
    sequence_data = get_sequence_bbox_data(sequence_id, gavd_dataset)
    
    if sequence_data is None:
        print(f"❌ No data found for sequence: {sequence_id}")
        return
    
    # Check video file
    input_video = Path(GAVD_SEQUENCES_DIR) / f"{sequence_id}.mp4"
    if not input_video.exists():
        print(f"❌ Input video not found: {input_video}")
        return
    
    # Get video properties for comparison
    cap = cv2.VideoCapture(str(input_video))
    if cap.isOpened():
        video_fps = int(cap.get(cv2.CAP_PROP_FPS))
        video_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cap.release()
    else:
        print(f"❌ Cannot open video for analysis")
        return
    
    # Display sequence information
    print(f"📋 Sequence Information:")
    print(f"  📼 Sequence ID: {sequence_id}")
    print(f"  📊 GAVD frames: {len(sequence_data)}")
    print(f"  🎯 GAVD frame range: {sequence_data['frame_num'].min()} - {sequence_data['frame_num'].max()}")
    print(f"  🎬 Video frames: {video_frames}")
    print(f"  ⚡ Video FPS: {video_fps}")
    
    if 'gait_pat' in sequence_data.columns:
        print(f"  🚶 Gait pattern: {sequence_data['gait_pat'].iloc[0]}")
    if 'cam_view' in sequence_data.columns:
        print(f"  📷 Camera view: {sequence_data['cam_view'].iloc[0]}")
    
    # Frame mapping analysis
    gavd_min = sequence_data['frame_num'].min()
    gavd_max = sequence_data['frame_num'].max()
    gavd_range = gavd_max - gavd_min + 1
    
    print(f"\\n🔍 Frame Mapping Analysis:")
    print(f"  📊 GAVD sequence length: {gavd_range} frames")
    print(f"  🎬 Video length: {video_frames} frames")
    print(f"  📐 Ratio: {gavd_range/video_frames:.2f} (GAVD/Video)")
    
    if gavd_range > video_frames:
        print(f"  ⚠️ GAVD frame range exceeds video length!")
        print(f"  💡 Will process all {video_frames} video frames with bounding box data")
    
    # Display bounding box statistics
    valid_bboxes = [bbox for bbox in sequence_data['bbox_parsed'] if bbox is not None]
    
    if valid_bboxes:
        print(f"\\n📐 Bounding Box Statistics:")
        print(f"  ✅ Valid bboxes: {len(valid_bboxes)}/{len(sequence_data)}")
        
        all_tops = [bbox['top'] for bbox in valid_bboxes]
        all_lefts = [bbox['left'] for bbox in valid_bboxes]
        all_widths = [bbox['width'] for bbox in valid_bboxes]
        all_heights = [bbox['height'] for bbox in valid_bboxes]
        
        print(f"  📏 Width range: {min(all_widths):.1f} - {max(all_widths):.1f} (avg: {np.mean(all_widths):.1f})")
        print(f"  📐 Height range: {min(all_heights):.1f} - {max(all_heights):.1f} (avg: {np.mean(all_heights):.1f})")
        print(f"  🎯 Position X: {min(all_lefts):.1f} - {max(all_lefts):.1f}")
        print(f"  🎯 Position Y: {min(all_tops):.1f} - {max(all_tops):.1f}")
        
        # Show first and last bbox
        print(f"\\n📦 Sample Bounding Boxes:")
        first_bbox = valid_bboxes[0]
        last_bbox = valid_bboxes[-1]
        
        print(f"  First frame: top={first_bbox['top']}, left={first_bbox['left']}, "
              f"width={first_bbox['width']}, height={first_bbox['height']}")
        print(f"  Last frame:  top={last_bbox['top']}, left={last_bbox['left']}, "
              f"width={last_bbox['width']}, height={last_bbox['height']}")
    else:
        print(f"❌ No valid bounding boxes found!")
        return
    
    print(f"\\n✂️ Testing Video Cropping:")
    print(f"  📥 Input: {input_video} ({input_video.stat().st_size / (1024*1024):.2f} MB)")
    
    # Clean up any existing test files
    test_output = Path(CROPPED_VIDEOS_DIR) / f"{sequence_id}_test_cropped.mp4"
    if test_output.exists():
        test_output.unlink()
        print(f"  🗑️ Removed existing test file")
    
    # Perform cropping
    success = crop_video_with_bbox(str(input_video), str(test_output), sequence_data)
    
    if success and test_output.exists():
        output_size_mb = test_output.stat().st_size / (1024*1024)
        input_size_mb = input_video.stat().st_size / (1024*1024)
        compression_ratio = output_size_mb / input_size_mb if input_size_mb > 0 else 0
        
        print(f"  📤 Output: {test_output} ({output_size_mb:.2f} MB)")
        print(f"  📊 Size reduction: {(1-compression_ratio)*100:.1f}% (ratio: {compression_ratio:.3f})")
        
        if output_size_mb > 0.01:  # Check if file has reasonable size
            print(f"  ✅ Cropping test SUCCESSFUL!")
            
            # Verify the output video can be opened
            test_cap = cv2.VideoCapture(str(test_output))
            if test_cap.isOpened():
                test_frames = int(test_cap.get(cv2.CAP_PROP_FRAME_COUNT))
                test_cap.release()
                print(f"  📊 Output video: {test_frames} frames")
            else:
                print(f"  ⚠️ Output video cannot be opened!")
        else:
            print(f"  ❌ Output file too small - cropping may have failed!")
            
    else:
        print(f"  ❌ Cropping test FAILED!")
    
    print("=" * 60)

# Clean up any existing test files first
existing_test_files = list(Path(CROPPED_VIDEOS_DIR).glob('*test_cropped.mp4'))
for test_file in existing_test_files:
    test_file.unlink()
    print(f"🗑️ Cleaned up: {test_file.name}")

# Run the test
test_bbox_cropping()


🗑️ Cleaned up: cljanb45y00083n6lmh1qhydd_test_cropped.mp4
🧪 Testing bounding box cropping for sequence: cljanb45y00083n6lmh1qhydd
📋 Sequence Information:
  📼 Sequence ID: cljanb45y00083n6lmh1qhydd
  📊 GAVD frames: 215
  🎯 GAVD frame range: 2532 - 2746
  🎬 Video frames: 214
  ⚡ Video FPS: 30
  🚶 Gait pattern: parkinsons
  📷 Camera view: left side
\n🔍 Frame Mapping Analysis:
  📊 GAVD sequence length: 215 frames
  🎬 Video length: 214 frames
  📐 Ratio: 1.00 (GAVD/Video)
  ⚠️ GAVD frame range exceeds video length!
  💡 Will process all 214 video frames with bounding box data
\n📐 Bounding Box Statistics:
  ✅ Valid bboxes: 215/215
  📏 Width range: 247.0 - 291.0 (avg: 268.1)
  📐 Height range: 485.0 - 504.0 (avg: 491.0)
  🎯 Position X: 453.0 - 805.0
  🎯 Position Y: 110.0 - 131.0
\n📦 Sample Bounding Boxes:
  First frame: top=129.0, left=805.0, width=247.0, height=485.0
  Last frame:  top=131.0, left=475.0, width=247.0, height=485.0
\n✂️ Testing Video Cropping:
  📥 Input: GAVD-sequences\cljanb45y0

In [13]:
# # Process sample videos with FIXED bounding box cropping
# print("🚀 Running sample processing with FIXED cropping...")
# print("=" * 70)

# sample_results = []

# for i, video_path in enumerate(sample_videos, 1):
#     print(f"\\n📹 Processing sample {i}/{len(sample_videos)}: {video_path.name}")
#     print("-" * 60)
    
#     # Check if we have bounding box data for this sequence
#     sequence_id = video_path.stem
#     if gavd_dataset is not None:
#         sequence_data = get_sequence_bbox_data(sequence_id, gavd_dataset)
#         if sequence_data is not None:
#             print(f"📋 Found {len(sequence_data)} bounding box annotations")
#             print(f"📊 GAVD frame range: {sequence_data['frame_num'].min()} - {sequence_data['frame_num'].max()}")
#             print(f"💡 Will ignore frame numbers and use bbox data for consistent cropping")
            
#             # Show sample bounding box
#             sample_bbox = sequence_data.iloc[0]['bbox_parsed']
#             if sample_bbox:
#                 print(f"📐 Sample bbox: top={sample_bbox['top']}, left={sample_bbox['left']}, "
#                       f"width={sample_bbox['width']}, height={sample_bbox['height']}")
#         else:
#             print("⚠️ No bounding box data found for this sequence")
    
#     # Clean any existing cropped video for this sequence to force re-cropping
#     cropped_path = Path(CROPPED_VIDEOS_DIR) / f"{sequence_id}_cropped.mp4"
#     if cropped_path.exists():
#         cropped_path.unlink()
#         print(f"🗑️ Removed existing cropped video to test new cropping")
    
#     success, output_path, error = process_video_with_hsmr(
#         video_path=video_path,
#         output_dir=OUTPUT_DIR,
#         gavd_df=gavd_dataset,  # Pass the GAVD dataset
#         use_bbox_crop=True,    # Enable bounding box cropping
#         verbose=True,
#         show_output=True  # Don't show HSMR output for cleaner test
#     )
    
#     sample_results.append({
#         'video': video_path.name,
#         'success': success,
#         'output': output_path,
#         'error': error
#     })
    
#     if success:
#         print(f"✅ Sample {i} completed successfully!")
#         if output_path and output_path.exists():
#             print(f"📊 Output file size: {output_path.stat().st_size / (1024*1024):.2f} MB")
#     else:
#         print(f"❌ Sample {i} failed: {error}")
    
#     print("-" * 60)

# # Summary
# print(f"\\n📋 Sample Processing Summary:")
# print(f"✅ Successful: {sum(1 for r in sample_results if r['success'])}")
# print(f"❌ Failed: {sum(1 for r in sample_results if not r['success'])}")

# for result in sample_results:
#     status = "✅" if result['success'] else "❌"
#     print(f"  {status} {result['video']}")
#     if not result['success']:
#         print(f"    Error: {result['error'][:100]}...")  # Truncate long errors

# # Show cropped videos info with detailed analysis
# cropped_videos = list(Path(CROPPED_VIDEOS_DIR).glob('*.mp4'))
# if cropped_videos:
#     print(f"\\n✂️ Generated {len(cropped_videos)} cropped videos:")
#     for cropped_video in cropped_videos:
#         size_mb = cropped_video.stat().st_size / (1024*1024)
#         print(f"  📄 {cropped_video.name} ({size_mb:.2f} MB)")
        
#         # Verify the cropped video can be opened
#         try:
#             test_cap = cv2.VideoCapture(str(cropped_video))
#             if test_cap.isOpened():
#                 frames = int(test_cap.get(cv2.CAP_PROP_FRAME_COUNT))
#                 width = int(test_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#                 height = int(test_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#                 test_cap.release()
#                 print(f"    ✅ Playable: {frames} frames, {width}x{height}")
#             else:
#                 print(f"    ❌ Cannot open video file")
#         except Exception as e:
#             print(f"    ❌ Error checking video: {e}")

# print("\\n🎉 FIXED cropping test complete!")


In [14]:
import gc  # For garbage collection

# Skip list management
SKIP_LIST_FILE = "failed_videos_skiplist.txt"

def load_skip_list():
    """Load the list of videos to skip from file."""
    skip_list = set()
    skip_file = Path(SKIP_LIST_FILE)
    if skip_file.exists():
        with open(skip_file, 'r') as f:
            skip_list = set(line.strip() for line in f if line.strip())
        print(f"📋 Loaded skip list: {len(skip_list)} videos to skip")
    return skip_list

def add_to_skip_list(video_name, error_msg):
    """Add a video to the skip list."""
    skip_file = Path(SKIP_LIST_FILE)
    with open(skip_file, 'a') as f:
        f.write(f"{video_name}\n")
    
    # Also log the detailed error
    error_log_file = Path("failed_videos_errors.log")
    with open(error_log_file, 'a', encoding='utf-8') as f:
        f.write(f"\n{'='*80}\n")
        f.write(f"Video: {video_name}\n")
        f.write(f"Timestamp: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Error: {error_msg}\n")
        f.write(f"{'='*80}\n")

def batch_process_videos(video_dir, output_dir, max_videos=None, skip_existing=True, 
                        use_bbox_crop=True, gavd_df=None, auto_skip_failed=True):
    """
    Batch process all videos in a directory with optional bounding box cropping.
    
    Args:
        video_dir (str): Directory containing input videos
        output_dir (str): Directory to save output files
        max_videos (int, optional): Maximum number of videos to process (for testing)
        skip_existing (bool): Whether to skip already processed videos
        use_bbox_crop (bool): Whether to use bounding box cropping
        gavd_df (pd.DataFrame): GAVD dataset with bounding box information
        auto_skip_failed (bool): Whether to automatically skip previously failed videos
    
    Returns:
        dict: Processing statistics and results
    """
    video_files = list(Path(video_dir).glob('*.mp4'))
    
    # Load skip list
    skip_list = load_skip_list() if auto_skip_failed else set()
    
    # Filter out videos in skip list
    if skip_list:
        original_count = len(video_files)
        video_files = [v for v in video_files if v.name not in skip_list]
        skipped_count = original_count - len(video_files)
        print(f"⏭️ Skipping {skipped_count} videos from previous failures")
    
    if max_videos:
        video_files = video_files[:max_videos]
    
    print(f"🎬 Starting batch processing of {len(video_files)} videos")
    print(f"📁 Input: {video_dir}")
    print(f"📁 Output: {output_dir}")
    print(f"⏭️ Skip existing: {skip_existing}")
    print(f"⚠️ Auto-skip failed: {auto_skip_failed}")
    print(f"✂️ Bounding box cropping: {'ENABLED' if use_bbox_crop and gavd_df is not None else 'DISABLED'}")
    print("=" * 60)
    
    results = []
    start_time = time.time()
    
    # Process with progress bar
    for i, video_path in enumerate(tqdm(video_files, desc="Processing videos"), 1):
        try:
            success, output_path, error = process_video_with_hsmr(
                video_path=video_path,
                output_dir=output_dir,
                gavd_df=gavd_df,  # Pass GAVD dataset
                use_bbox_crop=use_bbox_crop,  # Enable/disable bbox cropping
                verbose=False,  # Reduce verbosity for batch processing
                show_output=True  # Don't show output for batch processing
            )
            
            results.append({
                'video': video_path.name,
                'success': success,
                'output': output_path,
                'error': error,
                'size_mb': video_path.stat().st_size / (1024*1024)
            })
            
            # Handle failed videos
            if not success:
                # Show full error message
                print(f"\n❌ [{i:4d}/{len(video_files):4d}] {video_path.name}")
                print(f"📄 FULL ERROR MESSAGE:")
                print("=" * 60)
                print(error)
                print("=" * 60)
                
                # Add to skip list for future runs
                add_to_skip_list(video_path.name, error)
                print(f"📝 Added {video_path.name} to skip list")
            
            # Print periodic success updates
            elif i % 10 == 0:
                print(f"✅ [{i:4d}/{len(video_files):4d}] {video_path.name}")
            
            # Garbage collection every 20 videos to prevent memory buildup
            if i % 20 == 0:
                gc.collect()
                
        except Exception as e:
            error_msg = f"Unexpected exception during processing: {str(e)}"
            print(f"\n💥 [{i:4d}/{len(video_files):4d}] {video_path.name}")
            print(f"📄 EXCEPTION:")
            print("=" * 60)
            print(error_msg)
            print("=" * 60)
            
            results.append({
                'video': video_path.name,
                'success': False,
                'output': None,
                'error': error_msg,
                'size_mb': video_path.stat().st_size / (1024*1024)
            })
            
            # Add to skip list
            add_to_skip_list(video_path.name, error_msg)
            print(f"📝 Added {video_path.name} to skip list")
    
    end_time = time.time()
    total_time = end_time - start_time
    
    # Calculate statistics
    successful = sum(1 for r in results if r['success'])
    failed = len(results) - successful
    total_size_mb = sum(r['size_mb'] for r in results)
    avg_time_per_video = total_time / len(results) if results else 0
    
    # Count cropped videos
    cropped_videos = list(Path(CROPPED_VIDEOS_DIR).glob('*.mp4'))
    
    # Final garbage collection
    gc.collect()
    
    stats = {
        'total_videos': len(results),
        'successful': successful,
        'failed': failed,
        'total_time_seconds': total_time,
        'avg_time_per_video': avg_time_per_video,
        'total_size_mb': total_size_mb,
        'cropped_videos': len(cropped_videos),
        'skipped_from_list': len(skip_list) if skip_list else 0,
        'results': results
    }
    
    return stats

# Test function definition
print("🔧 Batch processing function defined successfully!")


🔧 Batch processing function defined successfully!


In [15]:

# Skip List Management Utilities
def show_skip_list_status():
    """Show current skip list status and recent failures."""
    skip_file = Path(SKIP_LIST_FILE)
    error_log_file = Path("failed_videos_errors.log")
    
    print("📋 Skip List Status:")
    print("=" * 50)
    
    if skip_file.exists():
        skip_list = load_skip_list()
        print(f"📄 Skip list file: {SKIP_LIST_FILE}")
        print(f"🚫 Videos to skip: {len(skip_list)}")
        
        if len(skip_list) > 0:
            print(f"📝 Recent entries (last 10):")
            with open(skip_file, 'r') as f:
                lines = f.readlines()
                for line in lines[-10:]:
                    print(f"  - {line.strip()}")
    else:
        print(f"📄 No skip list found ({SKIP_LIST_FILE})")
    
    print()
    if error_log_file.exists():
        size_mb = error_log_file.stat().st_size / (1024*1024)
        print(f"📄 Error log file: failed_videos_errors.log ({size_mb:.2f} MB)")
        print(f"💡 Check this file for detailed error messages")
    else:
        print(f"📄 No error log found")
    
    print("=" * 50)

def clear_skip_list():
    """Clear the skip list (use with caution!)."""
    skip_file = Path(SKIP_LIST_FILE)
    if skip_file.exists():
        skip_file.unlink()
        print(f"🗑️ Cleared skip list: {SKIP_LIST_FILE}")
    else:
        print(f"📄 No skip list to clear")

def remove_from_skip_list(video_names):
    """Remove specific videos from skip list."""
    skip_file = Path(SKIP_LIST_FILE)
    if not skip_file.exists():
        print(f"📄 No skip list found")
        return
    
    # Read current skip list
    with open(skip_file, 'r') as f:
        current_list = set(line.strip() for line in f if line.strip())
    
    # Remove specified videos
    if isinstance(video_names, str):
        video_names = [video_names]
    
    removed_count = 0
    for video_name in video_names:
        if video_name in current_list:
            current_list.remove(video_name)
            removed_count += 1
            print(f"✅ Removed {video_name} from skip list")
        else:
            print(f"⚠️ {video_name} not found in skip list")
    
    # Write back the updated list
    if removed_count > 0:
        with open(skip_file, 'w') as f:
            for video_name in sorted(current_list):
                f.write(f"{video_name}\n")
        print(f"📝 Updated skip list: removed {removed_count} videos")

# Show current status
show_skip_list_status()


📋 Skip List Status:
📋 Loaded skip list: 82 videos to skip
📄 Skip list file: failed_videos_skiplist.txt
🚫 Videos to skip: 82
📝 Recent entries (last 10):
  - cljap5o8u004r3n6llsoy3aww.mp4
  - cljapao5600503n6lhrbr4zii.mp4
  - cljapbtwe00543n6lsdbwbhhi.mp4
  - cljaotzi6002g3n6ljofm5j6d.mp4
  - cljaouueo002k3n6l57zl7081.mp4
  - cljaqc7jf00943n6l9cmdtsmn.mp4
  - cljaqdekt00983n6ldn9m222j.mp4
  - cljaqocwq00a23n6lj8kgw100.mp4
  - cljaqqdar00aa3n6lblt5iei7.mp4
  - cljaqrnsq00ae3n6lo53132n5.mp4

📄 Error log file: failed_videos_errors.log (0.38 MB)
💡 Check this file for detailed error messages


In [ ]:
# Run batch processing with bounding box cropping
# WARNING: This will process ALL videos in GAVD-sequences directory
# Set max_videos to a small number for testing, or None to process all

# For testing: process only 10 videos with bounding box cropping
# batch_stats = batch_process_videos(GAVD_SEQUENCES_DIR, OUTPUT_DIR, max_videos=10, 
#                                  use_bbox_crop=True, gavd_df=gavd_dataset)

# For full processing: remove max_videos parameter or set to None
batch_stats = batch_process_videos(GAVD_SEQUENCES_DIR, OUTPUT_DIR, max_videos=None,
                                 use_bbox_crop=True, gavd_df=gavd_dataset, 
                                 auto_skip_failed=True)  # Enable auto-skip of failed videos

print("\\n" + "="*60)
print("📊 BATCH PROCESSING COMPLETE!")
print("="*60)
print(f"📼 Total videos processed: {batch_stats['total_videos']}")
print(f"✅ Successful: {batch_stats['successful']}")
print(f"❌ Failed: {batch_stats['failed']}")
print(f"⏭️ Skipped from skip list: {batch_stats.get('skipped_from_list', 0)}")
print(f"✂️ Cropped videos generated: {batch_stats.get('cropped_videos', 0)}")
print(f"⏱️ Total time: {batch_stats['total_time_seconds']:.2f} seconds ({batch_stats['total_time_seconds']/60:.1f} minutes)")
print(f"⚡ Average time per video: {batch_stats['avg_time_per_video']:.2f} seconds")
print(f"💾 Total input size: {batch_stats['total_size_mb']:.1f} MB")

if batch_stats['failed'] > 0:
    print(f"\\n❌ Failed videos:")
    failed_videos = [r for r in batch_stats['results'] if not r['success']]
    for failed in failed_videos[:10]:  # Show first 10 failures
        print(f"  - {failed['video']}: {failed['error'][:80]}...")
    if len(failed_videos) > 10:
        print(f"  ... and {len(failed_videos) - 10} more failures")

print(f"\\n📁 Output files saved to: {OUTPUT_DIR}")
output_files = list(Path(OUTPUT_DIR).glob('*.npy'))
print(f"📄 Generated {len(output_files)} .npy files")

print(f"\\n📁 Cropped videos saved to: {CROPPED_VIDEOS_DIR}")
cropped_videos = list(Path(CROPPED_VIDEOS_DIR).glob('*.mp4'))
if cropped_videos:
    total_cropped_size = sum(v.stat().st_size for v in cropped_videos) / (1024*1024)
    print(f"✂️ Generated {len(cropped_videos)} cropped videos ({total_cropped_size:.1f} MB total)")
else:
    print("✂️ No cropped videos generated")


📋 Loaded skip list: 82 videos to skip
⏭️ Skipping 82 videos from previous failures
🎬 Starting batch processing of 1719 videos
📁 Input: ./GAVD-sequences
📁 Output: ./GAVD-hsmr-params
⏭️ Skip existing: True
⚠️ Auto-skip failed: True
✂️ Bounding box cropping: ENABLED


Processing videos:   0%|          | 1/1719 [02:50<81:30:38, 170.80s/it]


❌ [   1/1719] cljaoyz1c003c3n6l84cbjwg9.mp4
📄 FULL ERROR MESSAGE:
Command failed with return code 3221225477\nSTDOUT: Found GAVD-cropped-sequences\cljaoyz1c003c3n6l84cbjwg9_cropped.mp4 is a file. It will be regarded as a video file.
\nSTDERR: [07/28 13:40:27][INFO] 🚚 Loading inputs from: GAVD-cropped-sequences\cljaoyz1c003c3n6l84cbjwg9_cropped.mp4, regarded as <video>.

100%|██████████| 111/111 [00:00<00:00, 3961.57it/s]
[07/28 13:40:27][INFO] 📦 Totally 111 images are loaded.
[07/28 13:40:27][INFO] 🧱 Building detector.
c:\Users\1nkas-Strix-4090-ll\miniconda3\envs\hsmr\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
[07/28 13:40:30][INFO] [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/ViTDet/COCO/cascade_mask_rcnn_vitdet_h/f328730692/model_final_f05665.

Processing videos:   0%|          | 8/1719 [42:34<274:44:35, 578.07s/it]

In [ ]:
sad

In [ ]:
import numpy as np

# Find and inspect a sample output file
output_files = list(Path(OUTPUT_DIR).glob('*.npy'))

if output_files:
    # Load the first available output file
    sample_file = output_files[0]
    print(f"🔍 Inspecting: {sample_file.name}")
    print(f"📊 File size: {sample_file.stat().st_size / (1024*1024):.2f} MB")
    
    try:
        # Load the data
        data = np.load(sample_file, allow_pickle=True)
        print(f"\\n📋 Data structure:")
        print(f"  Type: {type(data)}")
        
        if isinstance(data, np.ndarray):
            print(f"  Shape: {data.shape}")
            print(f"  Dtype: {data.dtype}")
            
            # If it's an array of dictionaries (typical HSMR output)
            if data.dtype == object and len(data) > 0:
                print(f"\\n🎬 Video has {len(data)} frames")
                
                # Inspect first frame
                first_frame = data[0]
                if isinstance(first_frame, dict):
                    print(f"\\n📦 First frame keys: {list(first_frame.keys())}")
                    
                    for key, value in first_frame.items():
                        if isinstance(value, np.ndarray):
                            print(f"  {key}: shape={value.shape}, dtype={value.dtype}")
                        else:
                            print(f"  {key}: {type(value)} - {value}")
                    
                    # Show some specific parameter details
                    if 'poses' in first_frame:
                        poses = first_frame['poses']
                        print(f"\\n🤸 Pose parameters:")
                        print(f"  Shape: {poses.shape}")
                        print(f"  Min/Max: {poses.min():.3f} / {poses.max():.3f}")
                        print(f"  Mean: {poses.mean():.3f}")
                    
                    if 'betas' in first_frame:
                        betas = first_frame['betas']
                        print(f"\\n👤 Shape parameters (betas):")
                        print(f"  Shape: {betas.shape}")
                        print(f"  Min/Max: {betas.min():.3f} / {betas.max():.3f}")
                    
                    if 'patch_cam_t' in first_frame:
                        cam_t = first_frame['patch_cam_t']
                        print(f"\\n📷 Camera translation:")
                        print(f"  Shape: {cam_t.shape}")
                        print(f"  Values: {cam_t}")
        
        print(f"\\n✅ Successfully inspected {sample_file.name}")
        
    except Exception as e:
        print(f"❌ Error loading file: {e}")

else:
    print("❌ No output files found. Run the processing cells first.")


🔍 Inspecting: HSMR-cljan9b4p00043n6ligceanyp.npy
📊 File size: 0.19 MB
\n📋 Data structure:
  Type: <class 'numpy.ndarray'>
  Shape: (511,)
  Dtype: object
\n🎬 Video has 511 frames
\n📦 First frame keys: ['patch_cam_t', 'poses', 'betas', 'bbx_cs']
  patch_cam_t: shape=(1, 3), dtype=float32
  poses: shape=(1, 46), dtype=float32
  betas: shape=(1, 10), dtype=float32
  bbx_cs: <class 'list'> - [array([268.77588, 372.10785, 497.57043], dtype=float32)]
\n🤸 Pose parameters:
  Shape: (1, 46)
  Min/Max: -1.031 / 2.851
  Mean: 0.119
\n👤 Shape parameters (betas):
  Shape: (1, 10)
  Min/Max: -0.093 / 0.210
\n📷 Camera translation:
  Shape: (1, 3)
  Values: [[-0.07694656  0.06684598 32.80605   ]]
\n✅ Successfully inspected HSMR-cljan9b4p00043n6ligceanyp.npy


In [22]:
# CORRECTED: Per-frame dynamic cropping function using GAVD bbox data
def crop_video_with_bbox_per_frame(input_video_path: str, output_video_path: str, 
                                  sequence_data: pd.DataFrame, padding: int = 20, 
                                  show_frame_details: bool = False) -> bool:
    """
    Crop video using per-frame bounding box data from GAVD dataset.
    Each frame uses its corresponding bounding box for cropping.
    
    Args:
        input_video_path (str): Path to input video
        output_video_path (str): Path to save cropped video
        sequence_data (pd.DataFrame): Sequence data with bounding boxes
        padding (int): Extra padding around bounding box
        show_frame_details (bool): Whether to show frame-by-frame details
        
    Returns:
        bool: Success status
    """
    try:
        # Open input video
        cap = cv2.VideoCapture(input_video_path)
        if not cap.isOpened():
            print(f"❌ Cannot open video: {input_video_path}")
            return False
        
        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        print(f"🎬 Input video: {total_frames} frames at {fps} FPS ({width}x{height})")
        print(f"📋 Sequence data: {len(sequence_data)} frames")
        
        # Create sorted list of bounding boxes by frame number
        valid_bboxes = []
        for _, row in sequence_data.iterrows():
            frame_num = row['frame_num']
            bbox = row['bbox_parsed']
            if bbox is not None:
                valid_bboxes.append({
                    'frame_num': frame_num,
                    'bbox': bbox
                })
        
        if not valid_bboxes:
            print("❌ No valid bounding boxes found!")
            return False
        
        # Sort by frame number
        valid_bboxes = sorted(valid_bboxes, key=lambda x: x['frame_num'])
        
        print(f"📦 Found {len(valid_bboxes)} valid bounding boxes")
        
        # Get GAVD frame range
        min_gavd_frame = valid_bboxes[0]['frame_num']
        max_gavd_frame = valid_bboxes[-1]['frame_num']
        gavd_frame_range = max_gavd_frame - min_gavd_frame + 1
        
        print(f"🎯 GAVD frame range: {min_gavd_frame} to {max_gavd_frame} ({gavd_frame_range} frames)")
        print(f"🎯 Video frame range: 0 to {total_frames-1} ({total_frames} frames)")
        print("✂️ Using PER-FRAME dynamic cropping (exact bbox per frame)")
        
        # Calculate maximum dimensions for consistent output size
        all_bboxes = [item['bbox'] for item in valid_bboxes]
        max_width = max(bbox['width'] for bbox in all_bboxes) + 2 * padding
        max_height = max(bbox['height'] for bbox in all_bboxes) + 2 * padding
        
        # Ensure dimensions don't exceed original video
        output_width = min(int(max_width), width)
        output_height = min(int(max_height), height)
        
        print(f"📏 Output video size: {output_width}x{output_height} (max bbox + padding)")
        
        # Setup output video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (output_width, output_height))
        
        if not out.isOpened():
            print(f"❌ Cannot create output video: {output_video_path}")
            cap.release()
            return False
        
        frames_written = 0
        
        if show_frame_details:
            print(f"\n📋 FRAME-BY-FRAME DETAILS:")
            print("=" * 80)
        
        # Process each video frame with corresponding bbox
        for video_frame_idx in range(total_frames):
            ret, frame = cap.read()
            if not ret:
                break
            
            # Map video frame index to GAVD data
            # Since GAVD frames might not start from 0, we need to interpolate
            if len(valid_bboxes) == total_frames:
                # Direct 1:1 mapping
                bbox_data = valid_bboxes[video_frame_idx]
            else:
                # Interpolate based on progress through video
                progress = video_frame_idx / max(1, total_frames - 1)
                bbox_idx = min(int(progress * (len(valid_bboxes) - 1)), len(valid_bboxes) - 1)
                bbox_data = valid_bboxes[bbox_idx]
            
            bbox = bbox_data['bbox']
            gavd_frame_num = bbox_data['frame_num']
            
            # Calculate crop region for this specific frame's bbox
            crop_top = max(0, int(bbox['top']) - padding)
            crop_left = max(0, int(bbox['left']) - padding)
            crop_bottom = min(int(bbox['top'] + bbox['height']) + padding, height)
            crop_right = min(int(bbox['left'] + bbox['width']) + padding, width)
            
            actual_crop_width = crop_right - crop_left
            actual_crop_height = crop_bottom - crop_top
            
            # Show frame details if requested
            if show_frame_details and video_frame_idx < 10:  # Show first 10 frames
                print(f"Frame {video_frame_idx:3d}: GAVD={gavd_frame_num}")
                print(f"  📦 BBox: top={bbox['top']:.1f}, left={bbox['left']:.1f}, "
                      f"width={bbox['width']:.1f}, height={bbox['height']:.1f}")
                print(f"  ✂️ Crop: ({crop_left},{crop_top}) to ({crop_right},{crop_bottom}) "
                      f"= {actual_crop_width}x{actual_crop_height}")
            
            # Crop the frame using this frame's specific bbox
            cropped_frame = frame[crop_top:crop_bottom, crop_left:crop_right]
            
            # Create output frame with consistent dimensions (centered)
            output_frame = np.zeros((output_height, output_width, 3), dtype=np.uint8)
            
            # Center the cropped frame in the output frame
            start_y = max(0, (output_height - actual_crop_height) // 2)
            start_x = max(0, (output_width - actual_crop_width) // 2)
            end_y = min(output_height, start_y + actual_crop_height)
            end_x = min(output_width, start_x + actual_crop_width)
            
            # Ensure we don't exceed boundaries
            crop_h = min(actual_crop_height, end_y - start_y)
            crop_w = min(actual_crop_width, end_x - start_x)
            
            output_frame[start_y:start_y+crop_h, start_x:start_x+crop_w] = cropped_frame[:crop_h, :crop_w]
            
            out.write(output_frame)
            frames_written += 1
        
        if show_frame_details and total_frames > 10:
            print(f"  ... (showing first 10 frames, total: {total_frames})")
            print("=" * 80)
        
        # Cleanup
        cap.release()
        out.release()
        
        print(f"✅ Per-frame cropped video saved: {frames_written} frames written")
        
        # Verify output file
        if frames_written == 0:
            print("⚠️ Warning: No frames were written to output file!")
            return False
        
        if frames_written != total_frames:
            print(f"⚠️ Warning: Expected {total_frames} frames, but wrote {frames_written}")
            
        return True
        
    except Exception as e:
        print(f"❌ Error cropping video: {e}")
        return False

print("🔧 Per-frame dynamic cropping function defined successfully!")
print("✂️ Uses exact per-frame bbox data from GAVD dataset")


🔧 Per-frame dynamic cropping function defined successfully!
✂️ Uses exact per-frame bbox data from GAVD dataset


In [23]:
def batch_crop_videos_only(video_dir, cropped_output_dir, gavd_df, max_videos=None, 
                          skip_existing=True, show_progress=True):
    """
    Batch crop all videos using bounding box data without HSMR processing.
    
    Args:
        video_dir (str): Directory containing input videos
        cropped_output_dir (str): Directory to save cropped videos
        gavd_df (pd.DataFrame): GAVD dataset with bounding box information
        max_videos (int, optional): Maximum number of videos to process (for testing)
        skip_existing (bool): Whether to skip already cropped videos
        show_progress (bool): Whether to show progress bar and detailed info
    
    Returns:
        dict: Cropping statistics and results
    """
    if gavd_df is None:
        print("❌ GAVD dataset not loaded. Cannot perform bounding box cropping.")
        return None
    
    # Get all video files
    video_files = list(Path(video_dir).glob('*.mp4'))
    
    if max_videos:
        video_files = video_files[:max_videos]
    
    print(f"✂️ Starting batch video cropping")
    print(f"📁 Input directory: {video_dir}")
    print(f"📁 Cropped output directory: {cropped_output_dir}")
    print(f"📼 Total videos to process: {len(video_files)}")
    print(f"⏭️ Skip existing: {skip_existing}")
    print("=" * 60)
    
    results = []
    start_time = time.time()
    
    # Create output directory
    os.makedirs(cropped_output_dir, exist_ok=True)
    
    # Process videos with optional progress bar
    iterator = tqdm(video_files, desc="Cropping videos") if show_progress else video_files
    
    for i, video_path in enumerate(iterator, 1):
        try:
            sequence_id = video_path.stem
            cropped_video_path = Path(cropped_output_dir) / f"{sequence_id}_cropped.mp4"
            
            # Skip if already exists and skip_existing is True
            if skip_existing and cropped_video_path.exists():
                if show_progress and not isinstance(iterator, tqdm):
                    print(f"⏭️ [{i:4d}/{len(video_files):4d}] Skipping {video_path.name} - already cropped")
                
                results.append({
                    'video': video_path.name,
                    'success': True,
                    'output': cropped_video_path,
                    'skipped': True,
                    'error': None,
                    'size_mb': video_path.stat().st_size / (1024*1024)
                })
                continue
            
            # Get sequence bounding box data
            sequence_data = get_sequence_bbox_data(sequence_id, gavd_df)
            
            if sequence_data is None:
                error_msg = f"No bounding box data found for sequence: {sequence_id}"
                if show_progress and not isinstance(iterator, tqdm):
                    print(f"⚠️ [{i:4d}/{len(video_files):4d}] {video_path.name} - {error_msg}")
                
                results.append({
                    'video': video_path.name,
                    'success': False,
                    'output': None,
                    'skipped': False,
                    'error': error_msg,
                    'size_mb': video_path.stat().st_size / (1024*1024)
                })
                continue
            
            # Perform per-frame cropping using GAVD bbox data
            success = crop_video_with_bbox_per_frame(
                str(video_path),
                str(cropped_video_path),
                sequence_data,
                padding=20,
                show_frame_details=False
            )
            
            if success:
                if show_progress and not isinstance(iterator, tqdm):
                    output_size = cropped_video_path.stat().st_size / (1024*1024)
                    print(f"✅ [{i:4d}/{len(video_files):4d}] {video_path.name} -> {output_size:.2f} MB")
                
                results.append({
                    'video': video_path.name,
                    'success': True,
                    'output': cropped_video_path,
                    'skipped': False,
                    'error': None,
                    'size_mb': video_path.stat().st_size / (1024*1024)
                })
            else:
                error_msg = "Cropping failed - see detailed output above"
                if show_progress and not isinstance(iterator, tqdm):
                    print(f"❌ [{i:4d}/{len(video_files):4d}] {video_path.name} - {error_msg}")
                
                results.append({
                    'video': video_path.name,
                    'success': False,
                    'output': None,
                    'skipped': False,
                    'error': error_msg,
                    'size_mb': video_path.stat().st_size / (1024*1024)
                })
            
            # Garbage collection every 50 videos
            if i % 50 == 0:
                gc.collect()
                
        except Exception as e:
            error_msg = f"Exception during cropping: {str(e)}"
            if show_progress and not isinstance(iterator, tqdm):
                print(f"💥 [{i:4d}/{len(video_files):4d}] {video_path.name} - {error_msg}")
            
            results.append({
                'video': video_path.name,
                'success': False,
                'output': None,
                'skipped': False,
                'error': error_msg,
                'size_mb': video_path.stat().st_size / (1024*1024)
            })
    
    end_time = time.time()
    total_time = end_time - start_time
    
    # Calculate statistics
    successful = sum(1 for r in results if r['success'])
    failed = sum(1 for r in results if not r['success'])
    skipped = sum(1 for r in results if r.get('skipped', False))
    newly_cropped = successful - skipped
    total_size_mb = sum(r['size_mb'] for r in results)
    avg_time_per_video = total_time / len(results) if results else 0
    
    # Final garbage collection
    gc.collect()
    
    stats = {
        'total_videos': len(results),
        'successful': successful,
        'failed': failed,
        'skipped': skipped,
        'newly_cropped': newly_cropped,
        'total_time_seconds': total_time,
        'avg_time_per_video': avg_time_per_video,
        'total_size_mb': total_size_mb,
        'results': results
    }
    
    return stats

print("🔧 Batch video cropping function defined successfully!")


🔧 Batch video cropping function defined successfully!


In [24]:
# Test the corrected per-frame cropping function
def test_per_frame_cropping(sequence_id="cljanb45y00083n6lmh1qhydd"):
    """Test the corrected per-frame bounding box cropping functionality."""
    
    if gavd_dataset is None:
        print("❌ GAVD dataset not loaded. Cannot test per-frame cropping.")
        return
    
    print(f"🧪 Testing PER-FRAME cropping for sequence: {sequence_id}")
    print("=" * 70)
    
    # Get sequence data
    sequence_data = get_sequence_bbox_data(sequence_id, gavd_dataset)
    
    if sequence_data is None:
        print(f"❌ No data found for sequence: {sequence_id}")
        return
    
    # Check video file
    input_video = Path(GAVD_SEQUENCES_DIR) / f"{sequence_id}.mp4"
    if not input_video.exists():
        print(f"❌ Input video not found: {input_video}")
        return
    
    print(f"📋 Sequence Information:")
    print(f"  📼 Sequence ID: {sequence_id}")
    print(f"  📊 GAVD frames: {len(sequence_data)}")
    print(f"  🎯 GAVD frame range: {sequence_data['frame_num'].min()} - {sequence_data['frame_num'].max()}")
    
    # Show bbox variation to demonstrate per-frame differences
    valid_bboxes = [bbox for bbox in sequence_data['bbox_parsed'] if bbox is not None]
    if len(valid_bboxes) >= 5:
        print(f"\n📦 Bounding Box Variation (showing per-frame changes):")
        for i in [0, len(valid_bboxes)//4, len(valid_bboxes)//2, 3*len(valid_bboxes)//4, -1]:
            bbox = valid_bboxes[i]
            frame_idx = "first" if i == 0 else "last" if i == -1 else f"frame {i}"
            print(f"  {frame_idx:>10}: top={bbox['top']:6.1f}, left={bbox['left']:6.1f}, "
                  f"width={bbox['width']:5.1f}, height={bbox['height']:5.1f}")
    
    print(f"\n✂️ Testing Per-Frame Video Cropping:")
    print(f"  📥 Input: {input_video} ({input_video.stat().st_size / (1024*1024):.2f} MB)")
    
    # Clean up any existing test files
    test_output = Path(CROPPED_VIDEOS_DIR) / f"{sequence_id}_per_frame_test.mp4"
    if test_output.exists():
        test_output.unlink()
        print(f"  🗑️ Removed existing test file")
    
    # Perform per-frame cropping with frame details
    success = crop_video_with_bbox_per_frame(
        str(input_video), 
        str(test_output), 
        sequence_data,
        padding=20,
        show_frame_details=True  # Show detailed frame-by-frame info
    )
    
    if success and test_output.exists():
        output_size_mb = test_output.stat().st_size / (1024*1024)
        input_size_mb = input_video.stat().st_size / (1024*1024)
        
        print(f"  📤 Output: {test_output} ({output_size_mb:.2f} MB)")
        print(f"  📊 Size change: {output_size_mb/input_size_mb:.2f}x original")
        
        if output_size_mb > 0.01:  # Check if file has reasonable size
            print(f"  ✅ Per-frame cropping test SUCCESSFUL!")
            
            # Verify the output video can be opened
            test_cap = cv2.VideoCapture(str(test_output))
            if test_cap.isOpened():
                test_frames = int(test_cap.get(cv2.CAP_PROP_FRAME_COUNT))
                test_width = int(test_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                test_height = int(test_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                test_cap.release()
                print(f"  📊 Output video: {test_frames} frames, {test_width}x{test_height}")
            else:
                print(f"  ⚠️ Output video cannot be opened!")
        else:
            print(f"  ❌ Output file too small - cropping may have failed!")
            
    else:
        print(f"  ❌ Per-frame cropping test FAILED!")
    
    print("=" * 70)

# Run the per-frame cropping test
print("🔄 Testing CORRECTED per-frame cropping...")
test_per_frame_cropping()


🔄 Testing CORRECTED per-frame cropping...
🧪 Testing PER-FRAME cropping for sequence: cljanb45y00083n6lmh1qhydd
📋 Sequence Information:
  📼 Sequence ID: cljanb45y00083n6lmh1qhydd
  📊 GAVD frames: 215
  🎯 GAVD frame range: 2532 - 2746

📦 Bounding Box Variation (showing per-frame changes):
       first: top= 129.0, left= 805.0, width=247.0, height=485.0
    frame 53: top= 127.3, left= 768.2, width=247.0, height=485.0
   frame 107: top= 112.6, left= 659.3, width=278.4, height=501.1
   frame 161: top= 121.8, left= 535.5, width=287.9, height=493.3
        last: top= 131.0, left= 475.0, width=247.0, height=485.0

✂️ Testing Per-Frame Video Cropping:
  📥 Input: GAVD-sequences\cljanb45y00083n6lmh1qhydd.mp4 (0.46 MB)
🎬 Input video: 214 frames at 30 FPS (1280x720)
📋 Sequence data: 215 frames
📦 Found 215 valid bounding boxes
🎯 GAVD frame range: 2532 to 2746 (215 frames)
🎯 Video frame range: 0 to 213 (214 frames)
✂️ Using PER-FRAME dynamic cropping (exact bbox per frame)
📏 Output video size: 331x54

In [25]:
# Run batch video cropping for all sequences
# This will create cropped videos for all sequences without running HSMR

# For testing: process only a few videos
# crop_stats = batch_crop_videos_only(GAVD_SEQUENCES_DIR, CROPPED_VIDEOS_DIR, gavd_dataset, 
#                                   max_videos=10, skip_existing=True, show_progress=True)

# For full processing: process all videos
crop_stats = batch_crop_videos_only(GAVD_SEQUENCES_DIR, CROPPED_VIDEOS_DIR, gavd_dataset, 
                                  max_videos=None, skip_existing=True, show_progress=True)

if crop_stats:
    print("\n" + "="*60)
    print("✂️ BATCH VIDEO CROPPING COMPLETE!")
    print("="*60)
    print(f"📼 Total videos processed: {crop_stats['total_videos']}")
    print(f"✅ Successfully cropped: {crop_stats['successful']}")
    print(f"❌ Failed to crop: {crop_stats['failed']}")
    print(f"⏭️ Skipped (already existed): {crop_stats['skipped']}")
    print(f"🆕 Newly cropped: {crop_stats['newly_cropped']}")
    print(f"⏱️ Total time: {crop_stats['total_time_seconds']:.2f} seconds ({crop_stats['total_time_seconds']/60:.1f} minutes)")
    print(f"⚡ Average time per video: {crop_stats['avg_time_per_video']:.2f} seconds")
    print(f"💾 Total input size: {crop_stats['total_size_mb']:.1f} MB")
    
    # Show failed videos if any
    if crop_stats['failed'] > 0:
        print(f"\n❌ Failed cropping videos:")
        failed_videos = [r for r in crop_stats['results'] if not r['success']]
        for failed in failed_videos[:10]:  # Show first 10 failures
            print(f"  - {failed['video']}: {failed['error']}")
        if len(failed_videos) > 10:
            print(f"  ... and {len(failed_videos) - 10} more failures")
    
    # Show cropped videos directory info
    print(f"\n📁 Cropped videos saved to: {CROPPED_VIDEOS_DIR}")
    cropped_videos = list(Path(CROPPED_VIDEOS_DIR).glob('*.mp4'))
    if cropped_videos:
        total_cropped_size = sum(v.stat().st_size for v in cropped_videos) / (1024*1024)
        print(f"✂️ Total cropped videos: {len(cropped_videos)} ({total_cropped_size:.1f} MB total)")
        
        # Show some statistics about the cropped videos
        sample_videos = cropped_videos[:5]
        print(f"\n📊 Sample cropped videos:")
        for video in sample_videos:
            size_mb = video.stat().st_size / (1024*1024)
            print(f"  📄 {video.name} ({size_mb:.2f} MB)")
        if len(cropped_videos) > 5:
            print(f"  ... and {len(cropped_videos) - 5} more")
    else:
        print("✂️ No cropped videos found")
        
    print(f"\n🎯 Success rate: {crop_stats['successful']/crop_stats['total_videos']*100:.1f}%")
else:
    print("❌ Cropping operation failed to initialize")


✂️ Starting batch video cropping
📁 Input directory: ./GAVD-sequences
📁 Cropped output directory: ./GAVD-cropped-sequences
📼 Total videos to process: 1801
⏭️ Skip existing: True


Cropping videos:   0%|          | 0/1801 [00:00<?, ?it/s]

🎬 Input video: 511 frames at 30 FPS (1280x720)
📋 Sequence data: 512 frames
📦 Found 512 valid bounding boxes
🎯 GAVD frame range: 1757 to 2268 (512 frames)
🎯 Video frame range: 0 to 510 (511 frames)
✂️ Using PER-FRAME dynamic cropping (exact bbox per frame)
📏 Output video size: 337x551 (max bbox + padding)


Cropping videos:   0%|          | 1/1801 [00:01<30:30,  1.02s/it]

✅ Per-frame cropped video saved: 511 frames written
🎬 Input video: 214 frames at 30 FPS (1280x720)
📋 Sequence data: 215 frames
📦 Found 215 valid bounding boxes
🎯 GAVD frame range: 2532 to 2746 (215 frames)
🎯 Video frame range: 0 to 213 (214 frames)
✂️ Using PER-FRAME dynamic cropping (exact bbox per frame)
📏 Output video size: 331x544 (max bbox + padding)


...

✅ Per-frame cropped video saved: 343 frames written
🎬 Input video: 1108 frames at 60 FPS (1920x1080)
📋 Sequence data: 1109 frames
📦 Found 1109 valid bounding boxes
🎯 GAVD frame range: 792 to 1900 (1109 frames)
🎯 Video frame range: 0 to 1107 (1108 frames)
✂️ Using PER-FRAME dynamic cropping (exact bbox per frame)
📏 Output video size: 354x760 (max bbox + padding)


Cropping videos:   3%|▎         | 54/1801 [00:42<1:29:46,  3.08s/it]

✅ Per-frame cropped video saved: 1108 frames written
🎬 Input video: 1225 frames at 60 FPS (1920x1080)
📋 Sequence data: 1226 frames
📦 Found 1226 valid bounding boxes
🎯 GAVD frame range: 1937 to 3162 (1226 frames)
🎯 Video frame range: 0 to 1224 (1225 frames)
✂️ Using PER-FRAME dynamic cropping (exact bbox per frame)
📏 Output video size: 355x728 (max bbox + padding)


Cropping videos:   3%|▎         | 54/1801 [00:43<23:29,  1.24it/s]  


KeyboardInterrupt: 